# It is a chemical plant?

Homework for Practical Deep Learning for Coders lesson 1

## Setup

In [ ]:
# noqa: ruff[F405]

from fastcore.all import *
from fastai.vision.all import *
from utils import search_images, download_and_show_image
from time import sleep

In [ ]:
%autoreload 2

Fix VSCode Jupyter extension not displaying output of learner

In [ ]:
from IPython.display import clear_output, DisplayHandle


def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)


DisplayHandle.update = update_patch

## Homework

### Get images of chemical plants and power plants

In [ ]:
chemical = search_images(term="chemical plant", max_images=1)

In [ ]:
power = search_images(term="power plant", max_images=1)

In [ ]:
download_and_show_image(url=chemical[0], image_name="chemical")

In [ ]:
download_and_show_image(url=power[0], image_name="power")

In [ ]:
searches = ["chemical plant", "power plant"]
download_path = Path("chemical_or_not")

In [ ]:
for search in searches:
    dest = download_path.joinpath(search)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest=dest, urls=search_images(term=f"{search} photo"))
    sleep(5)
    download_images(dest=dest, urls=search_images(term=f"{search} day photo"))
    sleep(5)
    download_images(dest=dest, urls=search_images(term=f"{search} night photo"))
    resize_images(
        path=download_path.joinpath(search),
        max_size=400,
        dest=download_path.joinpath(search),
    )

In [ ]:
failed_images = verify_images(get_image_files(download_path))
[Path.unlink(img) for img in failed_images]
len(failed_images)

### Train model

In [ ]:
data_loaders = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=1337),
    get_y=parent_label,
    item_tfms=[Resize(192, method="squish")],
).dataloaders(download_path)

In [ ]:
data_loaders.show_batch(max_n=6)

In [ ]:
learner = vision_learner(dls=data_loaders, arch=resnet18, metrics=error_rate)
learner.fine_tune(3)

In [ ]:
learner.lr_find()

In [ ]:
learner.fine_tune(1, 1.4454397387453355e-05)

### Predict previously downloaded images

In [ ]:
is_chemical, _, probs = learner.predict(PILImage.create("chemical.jpg"))

In [ ]:
print(f"The image above is a {is_chemical}", f"I am {probs[0]:.2f} sure!", sep="\n")

### Export model

In [ ]:
learner.export(Path("models/chemical.pkl"))